# Fooocus (локальная копия + ваши модели) в Google Colab

Запуск **вашего** Fooocus с папки в Google Drive, со всеми подгруженными моделями.

## Перед запуском

1. **Залейте папку Fooocus в Google Drive** (см. `GOOGLE_COLAB_LOCAL_FOOOCUS.md`).
2. **Runtime → Change runtime type → GPU** (T4).
3. В ячейке ниже укажите путь к папке Fooocus на Drive (по умолчанию `MyDrive/Fooocus`).
4. Запустите ячейки по порядку.

In [ ]:
# Путь к папке Fooocus на Google Drive (относительно "Мой диск")
# Примеры: "Fooocus"  или  "AI-modeli/Fooocus"  или  "Мои проекты/AI-modeli/Fooocus"
DRIVE_FOOOCUS_PATH = "Fooocus"

In [ ]:
# Подключаем Google Drive и проверяем путь к Fooocus
from google.colab import drive
import os

drive.mount("/content/drive", use_metadata_cache=False)

FOOCUS_ROOT = os.path.join("/content/drive/MyDrive", DRIVE_FOOOCUS_PATH.strip("/"))
if not os.path.isdir(FOOCUS_ROOT):
    raise FileNotFoundError(
        f"Папка Fooocus не найдена: {FOOCUS_ROOT}\n"
        "Проверьте DRIVE_FOOOCUS_PATH и что папка загружена в Google Drive."
    )
print(f"Fooocus найден: {FOOCUS_ROOT}")

# Проверяем наличие чекпоинтов (моделей)
checkpoints_default = os.path.join(FOOCUS_ROOT, "models", "checkpoints")
checkpoints_alt = os.path.join(FOOCUS_ROOT, "models 12.07.29", "checkpoints")
if os.path.isdir(checkpoints_alt):
    path_checkpoints = checkpoints_alt
elif os.path.isdir(checkpoints_default):
    path_checkpoints = checkpoints_default
else:
    path_checkpoints = None

if path_checkpoints:
    models = [f for f in os.listdir(path_checkpoints) if f.endswith((".safetensors", ".ckpt"))]
    print(f"Найдено моделей в checkpoints: {len(models)}")
    for m in models[:10]:
        print(f"  - {m}")
    if len(models) > 10:
        print(f"  ... и ещё {len(models) - 10}")
    models_base = "models 12.07.29" if "12.07.29" in path_checkpoints else "models"
else:
    print("Внимание: папка checkpoints не найдена (ни models/checkpoints, ни models 12.07.29/checkpoints).")
    print("Fooocus при запуске может скачать модель по умолчанию.")
    models_base = "models"

In [ ]:
# Прописываем пути к моделям в config.txt для Colab (чтобы не зависеть от путей с Mac)
import json

config_path = os.path.join(FOOCUS_ROOT, "config.txt")
models_dir = os.path.join(FOOCUS_ROOT, models_base)
paths = {
    "path_checkpoints": [path_checkpoints] if path_checkpoints else [os.path.join(FOOCUS_ROOT, "models", "checkpoints")],
    "path_loras": [os.path.join(models_dir, "loras")],
    "path_embeddings": os.path.join(models_dir, "embeddings"),
    "path_vae_approx": os.path.join(models_dir, "vae_approx"),
    "path_vae": os.path.join(models_dir, "vae"),
    "path_outputs": os.path.join(FOOCUS_ROOT, "outputs"),
}

for key, val in paths.items():
    if isinstance(val, list):
        for v in val:
            os.makedirs(v, exist_ok=True)
    else:
        os.makedirs(val, exist_ok=True)

# Читаем существующий config, если есть, и обновляем только пути
if os.path.isfile(config_path):
    with open(config_path, "r", encoding="utf-8") as f:
        config = json.load(f)
else:
    config = {}
config.update(paths)

with open(config_path, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=4, ensure_ascii=False)
print("config.txt обновлён для Colab.")

In [ ]:
# Устанавливаем зависимости и запускаем Fooocus (без git — запуск через launch.py)
import os
os.chdir(FOOCUS_ROOT)  # все следующие команды выполняются из папки Fooocus на Drive

!pip install -q pygit2==1.15.1

# Запуск: launch.py сам поставит torch и requirements при первом запуске
# --share — публичная ссылка Gradio, --always-high-vram — под T4
!python launch.py --share --always-high-vram